# 近似最近鄰 Approximation Nearest Neighbors

這邊想討論的是，當商品的數量大到一個**等級**(可能是需要十萬以上..)的時候，去比對每個點來找最近鄰的方法會非常的低效。用例子來說明原理還是最好的:

如果有500萬個商品，每個商品的特徵向量表示$\in \textbf{R}^{100}$，我們想要尋找其中一個商品最鄰近的前十個。

In [1]:
import numpy as np
from tqdm import tqdm

items = int(5*1e6) #500萬
factor = 100
x = np.random.rand(items,factor) ## 記憶體~4GB

class TopRelated:
    ## 利用向量內積，查找最鄰近的物品(cosine based)
    def __init__(self, items_factors):
        ## 初始化需要正規化物品向量
        norms = np.linalg.norm(items_factors, axis=1)
        self.factors = items_factors / norms[:, np.newaxis]

    def get_related(self, itemid, N=10):
        scores = self.factors.dot(self.factors[itemid]) # cosine 
        best = np.argpartition(scores, -N)[-N:] # partion --> 小於此的放在左側
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])

In [44]:
top_related = TopRelated(x)
%time top_related.get_related(100)

Wall time: 266 ms


[(100, 1.0),
 (2266144, 0.85922529002190373),
 (1367840, 0.85190470560078513),
 (4827941, 0.85175188853230699),
 (4158610, 0.85165288118765758),
 (3025845, 0.85092234565859126),
 (4118586, 0.85018339949723798),
 (2610570, 0.84982956620805683),
 (333249, 0.84958760878590689),
 (4265367, 0.84917002416086451)]

對每個item計算最鄰近的10個item需要耗時280ms,估計對500萬個items計算需要至少 380hr

In [23]:
round(items *0.28 /3600,2)# hour

388.89

如果我們需要對海量的商品尋找最接近的，透過線性的查找(暴力法)會非常耗時。這時候可以透過近似近鄰(ANN)方法來幫助...

* 先看怎麼使用套件[annoy](https://github.com/spotify/annoy)。套件作者是超級大大(自己google一下)這個專案有**三千多星!!!!**

* 熟悉一下用法，之後會講原理。

In [20]:
import annoy
class ApproximateTopRelated:
    def __init__(self, items_factors, treecount=20):
        index = annoy.AnnoyIndex(items_factors.shape[1], 'angular')
        for i, row in tqdm(enumerate(items_factors)):
            index.add_item(i, row)
        index.build(treecount)
        self.index = index

    def get_related(self, itemid, N=10):
        neighbours = self.index.get_nns_by_item(itemid, N)
        return sorted(((other, 1 - self.index.get_distance(itemid, other))
                      for other in neighbours), key=lambda x: -x[1])

In [9]:
## 不要在自己的本機/筆電執行，耗用8G以上記憶體
%time app_top_related = ApproximateTopRelated(x)

5000000it [01:08, 72851.61it/s]


Wall time: 13min 15s


annoy使用的時候需要先建立一個索引二元樹，然後根據此樹作
1. 查找
2. 意外分支
3. random tree

建立這個index耗時13分15秒，而透過建立出來的index來查找最近的物品，幾乎是不太耗時的。

In [10]:
%time app_top_related.get_related(100)

Wall time: 0 ns


[(100, 1.0),
 (3164034, 0.4320356249809265),
 (1034936, 0.42606252431869507),
 (118226, 0.40478748083114624),
 (3624813, 0.4044606685638428),
 (2921124, 0.39136600494384766),
 (847370, 0.38512957096099854),
 (4961798, 0.37953001260757446),
 (122976, 0.37684887647628784),
 (416312, 0.37207216024398804)]

In [51]:
t0 = time.time()
for i in range(items):
    app_top_related.get_related(i)
t1 = time.time()
print('time elapse :{}s'.format(t1-t0))

time elapse :999.8614735603333s


與之前相比500萬個商品查找所有的（近似）最近鄰只需要耗時約`13 + 16 = 29 分`與直接計算所有商品間的內積並排序，效能上至少相差數百倍。

# 近似最近鄰
抄襲自annoy作者的[blog](https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html),可以的話看原文講的非常清楚，這裡只是做個中文筆記而已。
____

## 建立二元樹

如果有資料點如下，
![ann1](./img/ann1.png)    
想要建立一種資料結構，使得搜尋最鄰近的點複雜度不高於$\mathcal{O}(\log{}n)$ 

1. 隨意任選兩點，切出一與兩點等距之超平面
    
2. 在新切出的二平面中，繼續任選兩點切出等具超平面
    
3. 持續切割平面直到，使任平面中所含的點不大於_k_

## 搜尋過程

初始的紅色小點代表查詢的位置
![ann2](./img/ann2.png)

1. 按照二元樹找尋最鄰近每個節點(node)的位置，此時查詢的長度不會超過$\log{}n$
2. 允許**例外**發生，使用priority Queue設計一閥值，大於此值node可以走不同的分支。

## 隨機樹

除此之外允許產生隨機的二元樹狀結構(random tree)

![ann3](./img/ann3.png)
(ps.看原始網頁有[動圖](https://erikbern.com/assets/2015/09/animated.gif))
1. 針對一個點同時搜尋不同的樹
> We can search all trees at the same time using one single priority queue.
2. 根據幾棵樹找到的鄰近點再做point-wise計算
3. 取前N名(topN)

# 實作3D模型資料

讀資料/清資料(again...)

In [8]:
import numpy as np 
import pandas as pd
import csv
import sys
from tqdm import tqdm
sys.path.append('../')
from rec_helper import *

In [9]:
df = pd.read_csv('../rec-a-sketch/model_likes_anon.psv',
                 sep='|',quotechar='\\',quoting=csv.QUOTE_MINIMAL)
print(df.count())
df.drop_duplicates(inplace=True)
print(df.count())
df = threshold_interaction(df,rowname='uid',colname='mid',row_min=5,col_min=10)
inter,uid_to_idx,idx_to_uid,mid_to_idx,idx_to_mid=df_to_spmatrix(df,'uid','mid')
train,test, user_idxs = train_test_split(inter,split_count=1,fraction=0.2)

modelname    632832
mid          632832
uid          632832
dtype: int64
modelname    632677
mid          632677
uid          632677
dtype: int64
Starting interactions info
Number of rows: 62583
Number of cols: 28806
Sparsity: 0.04%
Ending interactions info
Number of rows: 13496
Number of columns: 13618
Sparsity: 0.25%


## implicit套件
用來計算items_embedding

1. 比較ann與暴力法效能
2. 人眼看一下相似性結果

In [11]:
import implicit
model = implicit.als.AlternatingLeastSquares(factors=50,regularization=0.01)
train64 = train.astype(np.float64)
model.fit(train64.T)

In [23]:
for i in tqdm(range(train.shape[1])):
    model.similar_items(i)

100%|██████████████████████████████████| 13618/13618 [00:04<00:00, 3350.27it/s]


In [28]:
approx_topRelated_item = ApproximateTopRelated(model.item_factors)
for i in tqdm(range(train.shape[1])):
    approx_topRelated_item.get_related(i)

13618it [00:00, 108942.13it/s]
100%|█████████████████████████████████| 13618/13618 [00:01<00:00, 12816.39it/s]


直接呼叫套件下的`similar_items`方法:
   > 3,350 items/s
   
使用近似鄰近法
 > 12,816 items/s

In [54]:
# approx_top_related_items = approx_topRelated_item.get_related(20)
# top_related_items = top_related.get_related(20)

In [67]:
import requests
from IPython.display import HTML, display
def get_thumbnails(top_related_items, idx, idx_to_mid, N=10):
#     row = sim[idx, :].A.ravel()
    topNitems,scores = zip(*top_related_items.get_related(idx))
    thumbs = []
    for x in topNitems:         
        response = requests.get('https://sketchfab.com/i/models/{}'.format(idx_to_mid[x])).json()
        thumb = [x['url'] for x in response['thumbnails']['images']]
#         print(thumb)
#         thumb = [x['url'] for x in response['thumbnails']['images'] if x['width'] == 200 and x['height']==200]
        if not thumb:
            print('no thumbnail')
        else:
            thumb = thumb[-2]
        thumbs.append(thumb)
    return thumbs

In [68]:
def display_item(thumbs,origin_id,N=5):
    try: 
        print('原圖======')
        thumb_html = '<img src='+ '\"'+thumbs[0]+'\">' 
        
    except TypeError:
        print('oops, 找不到小圖!!!')
        response = requests.get('https://sketchfab.com/i/models/{}'.format(idx_to_mid[origin_id])).json()
        thumb = [x['url'] for x in response['thumbnails']['images']][-2]
        thumb_html = '<img src= "{}"/>'.format(thumb)
        print('稍大的圖====')
    for url in thumbs[1:]:        
        if url:
            thumb_html += """ <img style='width:120px;margin:0px;float:left;border:1px solid black;' src='{}' />""".format(url)            
    return thumb_html

In [69]:
itemid = 0

related_items_url = get_thumbnails(top_related,itemid,idx_to_mid)
approx_related_items_url = get_thumbnails(approx_topRelated_item,itemid,idx_to_mid)

In [70]:
print('暴力法--------')
HTML(display_item(related_items_url,itemid))

暴力法--------
原圖======


In [71]:
print('近似法--------')
HTML(display_item(approx_related_items_url,itemid))

近似法--------
原圖======


# 小結
結果來看近似的結果與實際差異並不太大，當然跟喵喵才觀察一兩個物品的有關係。如果商品的數量很大，在乎在線上搜尋的即時性，可以改用近似近鄰法加速搜尋，不過在小樣本如果更在乎準確性，大可不必使用這個技巧。

另外如果我們要對用戶做推薦，勢必會計算向量內積$\textbf{X}_u \cdot \textbf{y}_i$然後對所有商品重排序，這個在線上也不太可行。因此微軟在弄[xbox遊戲推薦](https://www.microsoft.com/en-us/research/publication/speeding-up-the-xbox-recommender-system-using-a-euclidean-transformation-for-inner-product-spaces/)的問題時，提出一種高效的手法。這部分有機會再談嚕...